In [1]:
import os
import torch
from peft import LoraConfig, get_peft_model, set_peft_model_state_dict
from transformers import (
    AutoModelForCausalLM,
)
from transformers import AutoTokenizer



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/envs/dev/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda121.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 121
CUDA SETUP: Loading binary /opt/conda/envs/dev/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda121.so...


/opt/conda/envs/dev/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /opt/conda/envs/dev did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/opt/conda/envs/dev/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/opt/conda/envs/dev/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/opt/conda/envs/dev/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/

In [2]:
torch.cuda.is_available()

True

In [10]:
model_name = "EleutherAI/pythia-2.8b"
#adapter_path = "./models/adapter_pythia-2.8b_ep50_bs4.pth"
adapter_path = "./models/adapter.pth"
max_memory = {0: "22GIB", "cpu": "30GB"}

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto") #, max_memory=max_memory)

# small fix to get gpt generation working
model.config.pad_token_id = model.config.eos_token_id

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [12]:
peft_config = LoraConfig(
    bias="none",
    task_type="CAUSAL_LM",
    inference_mode=True,
    r=8, # better save and export the peft config used during training
    lora_alpha=16,
    lora_dropout=0.1,
)

model = get_peft_model(model, peft_config)

In [13]:
adapter_weights = torch.load(adapter_path, map_location="cuda")
set_peft_model_state_dict(model, adapter_weights)

In [14]:
model = model.to("cuda")

def predict(x, num_samples=1):
    with torch.no_grad():
        with torch.autocast("cuda"):
            golden_sample = f"Write a modern and engaging job posting for the following basic qualifications: {x}\r\nResponse: \r\n"

            inputs = tokenizer(golden_sample, return_tensors="pt")
            outputs = model.generate(
                input_ids=inputs["input_ids"].to("cuda"),
                do_sample=True,
                temperature=0.9,
                # num_beams=5,
                max_length=1024,
                num_return_sequences=num_samples,
            )

            responses = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)
            
            for response in responses:
                print(os.linesep.join(response.split("\r\n")))
                print("-----------------------------------------------------------------")
        

In [19]:
requirements = """
- You will work for StepStone and build revolutionary machine learning applications
- You will redefine job search and job discovery experience for millions of users
- You will bring Generative AI to the broad masses, greating great job experiences.
"""

requirements = "\r\n".join(requirements.split(os.linesep))
predict(requirements)

Write a modern and engaging job posting for the following basic qualifications: 
- You will work for StepStone and build revolutionary machine learning applications
- You will redefine job search and job discovery experience for millions of users
- You will bring Generative AI to the broad masses, greating great job experiences.

Response: 
Job description
- Are you seeking an opportunity to make a big impact on the future of search? We are looking for an experienced Sr. Software Engineering Engineer to join the Search & Discovery team, focused on delivering highly relevant and relevant job searches, improving the selection of applications that users get to interact with job assistance services and building out next- generation machine learning applications.
- Are you seeking an opportunity to make a big impact on the future of job searching? Do you want to redefine how job seekers and recruiters find each other? We enable customers to find each other through a dynamic online job searc

In [16]:
requirements = """
- You want to be part of revolutionizing shoes
- You are an expert in Shoe AI
- You wear shoes all day long
- You love shoes
- You want to change the future with Amazon Shoes AI
- You are an expert in smart business shoes powered by Amazon Alexa
- You will work on the development of next generation smart shoes
- Proven track record developing next generation shoe applications
"""

requirements = "\r\n".join(requirements.split(os.linesep))
predict(requirements, num_samples=1)

Write a modern and engaging job posting for the following basic qualifications: 
- You want to be part of revolutionizing shoes
- You are an expert in Shoe AI
- You wear shoes all day long
- You love shoes
- You want to change the future with Amazon Shoes AI
- You are an expert in smart business shoes powered by Amazon Alexa
- You will work on the development of next generation smart shoes
- Proven track record developing next generation shoe applications

Response: 
We're looking for talented, focused, and hard working Software Engineers who have experience building innovative, mission critical, high volume application systems. If you're passionate about developing cutting edge technologies, and want to have a significant impact on customers, this role may be for you. Our team members have experience in artificial intelligence, machine learning, electronic engineering, and computing across the full stack. We are looking for team members that combine these areas to further the developm